# Trajectory analysis
Contact matrix frame-wise

In [ ]:
nchains = 2000
concentration = 13
replica_number = 1

In [10]:
# The interest is on the sigmoid and the automatic part is the reading of the trajectory, select the frames of the aggregation start and also keep the maxclust.

# 1 Select the frames

%cd make_fibril_2000_13mM_300K_1/analysis_test

%ls

[Errno 2] No such file or directory: 'make_fibril_2000_13mM_300K_1/analysis_test'
/media/data/emanuele/TTR/greta_test_cutoff_55_ex_3/epsilon_0295_c12_Native2reweight_noN1_gd0/concentration_gradient/make_fibril_2000_13mM_300K_1/analysis_test
13mM_histo_dt-10000.png           matrix-dt10000.dat
13mM_matrix_dt-10000.png          maxclust.ndx
2000_pep-13mM.gro@                maxclust.xvg
NOPBC_prod_2000-13mM-dt1000.xtc@  nclust.xvg
avclust.xvg                       plot_histo_norm_dt10000.txt
csize.xpm                         plot_histogram_dt10000.txt
csizew.xpm                        plot_matrix_dt10000.txt
histo-clust.xvg                   prod_2000-13mM-dt10000.xtc
last_frame.ndx                    prod_2000-13mM.tpr@
last_frame.xtc                    temp.xvg


In [30]:
import pandas as pd
import numpy as np

def make_plots(matrix):
    #print(matrix_dict)

    # Changing the index so now it is in picoseconds
    #matrix[0] = matrix[0].apply(lambda x:x*10)
    #matrix = matrix.set_index([0])

    ### Data handling
    #### Defining the Maxclust and the number of frames
    clustsize = list(range(0, len(matrix.columns), 1))
    frames = list(range(0, len(matrix.index), 1))
    fibril = list(range(2, len(matrix.columns)))
    #not_fibril = list(range(3, 10, 1))


    #### Multiply the columns for its clustsize
    size = pd.Series(clustsize)
    matrix = matrix.mul(size, axis = 1)

    #### Replace 0 with n.an
    matrix[matrix == 0] = np.nan

    #### I don't care about monomers
    matrix = matrix.drop([0, 1], axis = 1)

    ### Make the histogram to compare with lab results
    # Set the not-fibril to 0 and have a sort of lag time 

# At the moment i keep also the matrix for multiple fibrils but now i focus on the normalized histogram

    histo_norm = matrix.copy()
    histo_norm[[2, 3, 4, 5, 6, 7, 8, 9, 10]] = 0
    histo_norm = histo_norm[fibril].sum(axis=1)

    
    # Matrix transpose
    #matrix = matrix.T
    matrix = [] # To save some RAM

    return matrix, histo_norm

In [35]:
import os, os.path

dat_file = list(filter(lambda k: '.dat' in k, os.listdir('.')))


# From a single element list to a string
dat_file = ''.join(dat_file)

matrix, histo_norm = make_plots(pd.read_csv(dat_file, sep = '\\s+', header = None))


histo_norm


0        0.0
1        0.0
2        0.0
3       13.0
4       20.0
       ...  
76    1961.0
77    1964.0
78    1964.0
79    1963.0
80    1962.0
Length: 81, dtype: float64

In [40]:
import mdtraj as md

trajectory_file = list(filter(lambda k: '.xtc' in k, os.listdir('.')))
trajectory_file = list(filter(lambda k: 'NOPBC' not in k, trajectory_file))
trajectory_file.remove('last_frame.xtc')
trajectory_file = ''.join(trajectory_file)

trajectory = md.load(trajectory_file, top = f'{nchains}_pep-{concentration}mM_{replica_number}.gro')

['prod_2000-13mM-dt10000.xtc']